In [72]:
import kagglehub
import pandas as pd
import numpy as np
import tensorflow as tf


# Download latest version
path = kagglehub.dataset_download("nelgiriyewithana/emotions")

print("Path to dataset files:", path)

Path to dataset files: /home/eugen/.cache/kagglehub/datasets/nelgiriyewithana/emotions/versions/1


In [73]:
dataset = pd.read_csv(path + '/text.csv')
print(dataset.head(10))
print(dataset['label'].value_counts())

   Unnamed: 0                                               text  label
0           0      i just feel really helpless and heavy hearted      4
1           1  ive enjoyed being able to slouch about relax a...      0
2           2  i gave up my internship with the dmrg and am f...      4
3           3                         i dont know i feel so lost      0
4           4  i am a kindergarten teacher and i am thoroughl...      4
5           5         i was beginning to feel quite disheartened      0
6           6  i would think that whomever would be lucky eno...      2
7           7  i fear that they won t ever feel that deliciou...      1
8           8  im forever taking some time out to have a lie ...      5
9           9  i can still lose the weight without feeling de...      0
label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64


In [74]:
labels_names = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'] # sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5).
labels = np.arange(6) # sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5).
dataset.drop('Unnamed: 0', axis=1, inplace=True)
dataset.head(1)

,text,label
0,i just feel really helpless and heavy hearted,4


In [75]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to /home/eugen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/eugen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/eugen/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [76]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    all = nltk.word_tokenize(text)
    filtered = [word for word in all if word.lower() not in stop_words]

    return ' '.join(filtered)

row_index = 10
print(dataset.iloc[row_index]['text'])
print(remove_stopwords(dataset.iloc[row_index]['text']))

i try to be nice though so if you get a bitchy person on the phone or at the window feel free to have a little fit and throw your pen at her face
try nice though get bitchy person phone window feel free little fit throw pen face


In [77]:
dataset['text'] = dataset['text'].apply(remove_stopwords)

In [78]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset['text'])

sequences = tokenizer.texts_to_sequences(dataset['text'])
sequences[2:3]

[[721, 5943, 40551, 2, 1013]]

In [79]:
sequences_padding = pad_sequences(sequences, maxlen=50, padding='post')
sequences_padding[2:3].size

50

In [81]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(50,)),
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6, activation='softmax'),
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) #'recall', 'precision'
model.fit(sequences_padding, dataset['label'])

2025-06-03 16:30:32.684906: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 83361800 exceeds 10% of free system memory.
I0000 00:00:1748957437.485078  112404 cuda_dnn.cc:529] Loaded cuDNN version 90300


13026/13026 ━━━━━━━━━━━━━━━━━━━━ 273s 20ms/step - accuracy: 0.8147 - loss: 0.4555


In [82]:
y_pred = model.predict(sequences_padding)
y_pred

2025-06-03 17:07:15.662511: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 83361800 exceeds 10% of free system memory.


13026/13026 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step


array([[4.5245242e-01, 1.7216944e-03, 2.4498662e-04, 1.0242596e-02,
        5.3296965e-01, 2.3686453e-03],
       [9.9892300e-01, 2.5420747e-04, 6.7114801e-05, 3.3271499e-04,
        3.6148346e-04, 6.1369778e-05],
       [5.1315692e-05, 1.0765918e-05, 2.6121493e-06, 1.7306284e-04,
        9.9924111e-01, 5.2119454e-04],
       ...,
       [2.1038904e-06, 1.5235541e-04, 1.1945422e-05, 5.3386048e-06,
        7.4932125e-04, 9.9907887e-01],
       [8.0703889e-05, 3.7057656e-05, 1.0052256e-05, 9.9945921e-01,
        2.8092985e-04, 1.3212301e-04],
       [1.9342213e-05, 7.4311293e-04, 7.9895784e-05, 4.3660159e-05,
        2.7349724e-03, 9.9637908e-01]], dtype=float32)

In [ ]:
dataset['predicted'] = y_pred
dataset.head(10)

,text,label,predicted
0,feel really helpless heavy hearted,4,0.452452
1,ive enjoyed able slouch relax unwind frankly n...,0,0.998923
2,gave internship dmrg feeling distraught,4,0.000051
